In [ ]:
pip install dash

In [ ]:
pip install jupyter_dash

In [ ]:
# Load the dataset
# df = pd.read_csv("sample_train.csv")

df = pd.read_csv("sample_train.csv")

# Convert date_time column to datetime
df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])

# Extract year information from the date-time column
df['year'] = df['trans_date_trans_time'].dt.year

# Create Dash app
app = dash.Dash(__name__)

# Define the layout with a menu and date pickers
app.layout = html.Div([
    html.H1("Interactive Dashboard"),

    # Menu
    html.Div([
        html.Label("Select Chart Type:"),
        dcc.Dropdown(
            id="chart-type",
            options=[
                {"label": "Pie-chart 1", "value": "pie-chart 1"},
                {"label": "Bar Chart", "value": "bar-chart"},
                {"label": "Pie Chart 2", "value": "pie-chart 2"},
                {"label": "Scatter Plot", "value": "scatter-plot"}
            ],
            value="histogram"
        ),
    ], style={"width": "250px", "margin-bottom": "20px"}),

    # Date pickers
    html.Div([
        html.Label("Select Date Range:"),
        dcc.DatePickerRange(
            id="date-range",
            start_date=df['trans_date_trans_time'].min(),
            end_date=df['trans_date_trans_time'].max(),
            display_format='YYYY-MM-DD'
        ),
    ], style={"width": "500px", "margin-bottom": "20px"}),

    # Placeholder for chart
    html.Div(id="chart-output"),
])

# Callback for updating the chart based on selected chart type and date range
@app.callback(Output("chart-output", "children"),
              Input("chart-type", "value"),
              Input("date-range", "start_date"),
              Input("date-range", "end_date"))
def update_chart(chart_type, start_date, end_date):
    filtered_df = df[(df["trans_date_trans_time"] >= start_date) & (df["trans_date_trans_time"] <= end_date)]
    if chart_type == "pie-chart 1":
        pie_chart_data = filtered_df["category"].value_counts().reset_index()
        fig = px.pie(pie_chart_data, values="category", names="index", title="Category Distribution")
    elif chart_type == "bar-chart":
        bar_chart_data = filtered_df.groupby("category").mean().reset_index()
        fig = px.bar(bar_chart_data, x="category", y="amt", color="category")
    elif chart_type == "pie-chart 2":
        pie_chart_data = filtered_df["gender"].value_counts().reset_index()
        fig = px.pie(pie_chart_data, values="gender", names="index", title="Gender Distribution")
    elif chart_type == "scatter-plot":
        fig = px.scatter(filtered_df, x="lat", y="long", color="is_fraud", title="Scatter Plot")
    return dcc.Graph(figure=fig)

if __name__ == "__main__":
    app.run_server(debug=False)


# if __name__ == "__main__":
#     app.run_server(mode='inline', debug=False)
